In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from math import sqrt
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Embedding, TimeDistributed, LeakyReLU
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot
from pickle import load
from numpy import *
from pandas import *
from kerastuner import HyperModel
from kerastuner.tuners import BayesianOptimization

In [2]:
X_train = np.load("X_train.npy", allow_pickle=True)
Y_train = np.load("y_train.npy", allow_pickle=True)
X_test = np.load("X_test.npy", allow_pickle=True)
Y_test = np.load("y_test.npy", allow_pickle=True)
Yc_train = np.load("yc_train.npy", allow_pickle=True)
Yc_test = np.load("yc_test.npy", allow_pickle=True)

In [3]:
class LSTMHyperModel(HyperModel):
    
    def __init__(self, input_dim,feature_size,output_dim):
        self.input_dim = input_dim
        self.feature_size = feature_size
        self.output_dim = output_dim
        
    def build(self, hp):
        model = Sequential()
        model.add(
            Bidirectional(LSTM(
                units=hp.Int('units_directional', min_value = 32,max_value =512,step = 32),
                activation='tanh',
                input_shape=(input_dim,feature_size)
                )
            )
        )
        
        model.add(
            Dense(
                units=hp.Int('units_dense', min_value = 32,max_value =512,step = 32),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(Dense(1))
        
        model.compile(
            optimizer=tensorflow.keras.optimizers.Adam(hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4,2e-2, 2e-3, 2e-4,3e-2, 3e-3, 3e-4,4e-2, 4e-3, 4e-4,5e-2, 5e-3, 5e-4])),loss='mse',metrics=['mse']
        )
        
        return model

In [4]:
input_dim = X_train.shape[1]
feature_size = X_train.shape[2]
output_dim = Y_train.shape[1]
hypermodel = LSTMHyperModel(input_dim,feature_size,output_dim)

In [5]:
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_loss',
            max_trials=10,
            seed=42,
            executions_per_trial=2
        )
tuner_bo.search(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), verbose=4)
best_model = tuner_bo.get_best_models(num_models=1)[0]
best_model.evaluate(X_test, Y_test)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
51/51 [==============================] - 20s 7ms/step - loss: 0.0095 - mse: 0.0095


[0.023471442982554436, 0.023471442982554436]

In [6]:
tuner_bo.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name='mse', direction='min')
Trial summary
Hyperparameters:
units_directional: 384
units_dense: 32
dense_activation: relu
learning_rate: 0.002
Score: 0.00026059716765303165
Trial summary
Hyperparameters:
units_directional: 128
units_dense: 128
dense_activation: relu
learning_rate: 0.0004
Score: 0.00027487549232319
Trial summary
Hyperparameters:
units_directional: 128
units_dense: 160
dense_activation: relu
learning_rate: 0.001
Score: 0.0002798866044031456
Trial summary
Hyperparameters:
units_directional: 352
units_dense: 64
dense_activation: relu
learning_rate: 0.005
Score: 0.00031532449065707624
Trial summary
Hyperparameters:
units_directional: 448
units_dense: 192
dense_activation: sigmoid
learning_rate: 0.001
Score: 0.0003377930406713858
Trial summary
Hyperparameters:
units_directional: 224
units_dense: 480
dense_activation: sigmoid
learning_rate: 0.0003
Score: 0.0003746415168279782
Trial summary
Hyperpar